In [ ]:
# default_exp colab

# colab
> helpers for colab

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from nicHelper.encrypt import encryptPasswordAes, decryptPasswordAes
from beartype import beartype
from getpass import getpass
from functools import partial
from typing import Tuple, Optional
from getpass import getpass
import subprocess, pickle, os

# save aws pw

In [ ]:
#export
@beartype
def saveAwsPw(awskey:bytes, awssecret:bytes,
              path:str = '/content/drive/MyDrive/.awskey', password:bytes = b''):
  '''
    save your aws password in your 
  '''

  # encrypt with aes
  encrypt = partial(encryptPasswordAes, password)
  cryptkey, keynonce = encrypt(awskey)
  cryptsecret, secretnonce = encrypt(awssecret)
  
  # save file as pickle
  print(f'saving the AES encrypted credentials to {path}')
  with open (path ,'wb') as f:
    pickle.dump((cryptkey, cryptsecret, keynonce, secretnonce),f)
  

In [ ]:
def test_saveAwsPw():
  # enter password == test ##
  saveAwsPw(awskey = b'test', awssecret=b'password', path = '/tmp/test', password = b'test')
  with open ('/tmp/test', 'rb') as f:
    r = pickle.load(f)
  print(r)
test_saveAwsPw()

saving the AES encrypted credentials to /tmp/test
(b'r\xab\x97\xdd', b'26\xd0\xec\xa7\x11\x9e\x90', b'\x9c\xe5\xd8\x9d\xfd"}vFb&\xf5%\xc1H\xda', b'P\xe9\x93\xecX\xc5\x80s8\xea.x)p\x93\xbb')


# load aws pw

In [ ]:
#export
@beartype
def loadAwsPw(path = '/content/drive/MyDrive/.awskey', 
              password:Optional[bytes] = b'' )->Optional[Tuple[bytes,bytes]]:
  '''
    load key and secret from path
  '''
  print( f'loading password from {path}')
  
  with open (path, 'rb') as f:
    cryptkey, cryptsecret, keynonce, secretnonce = pickle.load(f)
  decryptedKey = decryptPasswordAes(cryptkey,password,keynonce)
  decryptedSecret = decryptPasswordAes(cryptsecret, password, secretnonce)
  return decryptedKey, decryptedSecret
  

In [ ]:
def test_loadAwsPw():
  r = loadAwsPw(path = '/tmp/test', password = b'test')
  print(r)
test_loadAwsPw()

loading password from /tmp/test
(b'test', b'password')


# set up aws

In [ ]:
#export
@beartype
def setUpAws(awsKey:str, awsSecret:str, 
             profile:Optional[str] = None, region:str = 'ap-southeast-1')->Tuple[str,str,str]:
  ''' 
    this code generate a string to be executed to the shell
    you can do this with, however, its not a bad idea to check before executing
    
    (exec(c) for c in setUpAws(...))
    
    response:
      (setupKey, setupSecret, setupRegion)
      
  '''
  if profile:  profileParameter = f'--profile {profile} '
  else: profileParameter = ''
    
  setupPrefix = f'aws configure {profileParameter}'
  setupKey = f'{setupPrefix}set aws_access_key_id {awsKey}'
  setupSecret = f'{setupPrefix}set aws_secret_access_key {awsSecret}'
  setupRegion = f'{setupPrefix}set default.region {region}'
  
  
  return setupKey, setupSecret, setupRegion
  

In [ ]:
def testSetUpAws():
  r = setUpAws('test', 'secret', 'testProfile')
  print(r)
testSetUpAws()

('aws configure --profile testProfile set aws_access_key_id test', 'aws configure --profile testProfile set aws_secret_access_key secret', 'aws configure --profile testProfile set default.region ap-southeast-1')


# autosetup aws

In [ ]:
#export
debugstring = lambda x: ' '.join(x.split(" ")[:-1]) + ' ' + '*'*len(x.split(" ")[-1])
def autoSetupAws(path, profile=None, region='ap-southeast-1', mockup = False, password:Optional[bytes] = None):
  key, secret = loadAwsPw(path, password=password) # key and secret in bytes
  try:
    key, secret = key.decode(), secret.decode()
  except:
    print('cant decode key and secret, maybe the password is wrong')
    return
    
#   print(key,secret)
  setupStrings = setUpAws(awsKey=key,
                          awsSecret=secret,
                          profile=profile,
                          region=region)
  # check if reunning in colab
  isColab =  'google.colab' in str(get_ipython())
  
  if isColab:
    print('Running on CoLab')
  else:
    print('Not running on colab')
    
    
  if mockup:
    return setupStrings
  for setupString in setupStrings:
    if isColab:
      print(f'executing {debugstring(setupString)}')
      os.system(setupString)
    else:
      print(f'Not running on CoLab{debugstring(setupString)}')
      print(setupString[:-5])
  
  

In [ ]:
def testAutoSetupAws():
  autoSetupAws(path = '/tmp/test', password=b'test')
  print('test successful')
  autoSetupAws(path = '/tmp/test', password=b'tst')
testAutoSetupAws()

loading password from /tmp/test
Not running on colab
Not running on CoLabaws configure set aws_access_key_id ****
aws configure set aws_access_key_id
Not running on CoLabaws configure set aws_secret_access_key ********
aws configure set aws_secret_access_key pas
Not running on CoLabaws configure set default.region **************
aws configure set default.region ap-southe
test successful
loading password from /tmp/test
cant decode key and secret, maybe the password is wrong
